In [0]:
display(dbutils.fs.ls("/Volumes/workspace/default/emp/EMP_FF_SRC.txt"))

In [0]:
display(dbutils.fs.ls("/Volumes/workspace/default/emp/"))

In [0]:
display(dbutils.fs.ls("/Volumes/workspace/default/emp/EMP_DELTA"))

In [0]:
display(dbutils.fs.ls("/Volumes/workspace/default/emp/EMP_FF_SRC.txt"))

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

df = spark.read.format("csv").option("header", "true").option("dateFormat", "MM/dd/yyyy").option("inferSchema", "true").option("delimiter", "|").load("/Volumes/workspace/default/emp/EMP_FF_SRC.txt")

#df = df.withColumn("HIRE_DATE", to_date(col("HIRE_DATE"), "MM/dd/yyyy"))
df.display()

In [0]:
df.write.partitionBy("DEPARTMENT_ID").mode("overwrite").format("delta").save("/Volumes/workspace/default/emp/EMP_DELTA")

In [0]:
df.repartition(5).write.mode("overwrite").format("parquet").save("/Volumes/workspace/default/emp/EMP_PARQUET")

In [0]:
from pyspark.sql import Row

In [0]:
df_data  = [Row(key= "A", value=1)]*100 + [Row(key= "B", value=2)]*2 + [Row(key= "C", value=3)]
df_a = spark.createDataFrame(df_data)
df_a.display()

In [0]:
df_bdata  = [Row(key= "A", name="Alice")] + [Row(key= "B", name="Bob")] + [Row(key= "C", name="Charles")]
df_B = spark.createDataFrame(df_bdata)
df_B.display()

In [0]:
df_salt = df_a.withColumn("salt", floor(rand(seed = 1)*3))
df_salt = df_salt.withColumn("salted_key", concat_ws("_",df_salt["key"],df_salt["salt"]))
df_salt.display()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
ary = array([lit(i) for i in range(3)])

df_B = df_B.withColumn("salt", explode(ary))
df_B.display()
df_B = df_B.withColumn("salted_key", concat_ws("_",df_B["key"],df_B["salt"]))
df_B.display()


In [0]:
joined_df = df_salt.join(df_B, df_salt.salted_key == df_B.salted_key, "inner").select(df_salt.key, df_salt.value, df_B.name)
joined_df.display()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

df_emp = spark.read.format("csv").option("header", "true").option("dateFormat", "MM/dd/yyyy").option("inferSchema", "true").option("delimiter", "|").load("/Volumes/workspace/default/emp/EMP_FF_SRC.txt")

df_emp.display()


df_dept = spark.read.format("csv").option("header", "true").option("dateFormat", "MM/dd/yyyy").option("inferSchema", "true").option("delimiter", ",").load("/Volumes/workspace/default/emp/src_DEPT_FF.csv")

df_dept.display()

#df = df.withColumn("HIRE_DATE", to_date(col("HIRE_DATE"), "MM/dd/yyyy"))


In [0]:
df_emp.write.partitionBy("DEPARTMENT_ID").mode("overwrite").format("csv").save("/Volumes/workspace/default/emp/emp_CSV_salt")


In [0]:
display(dbutils.fs.ls("/Volumes/workspace/default/emp/emp_CSV_salt"))

In [0]:
df_emp_salt = df_emp.withColumn("salt", floor(rand(seed = 1)* 3))

df_emp_salt = df_emp_salt.withColumn("salted_key", concat_ws("-",df_emp_salt["DEPARTMENT_ID"],df_emp_salt["salt"]))
df_emp_salt.display()

In [0]:
ary = array([lit(i) for i in range(3)])
df_dept_salt = df_dept.withColumn("salt", explode(ary))

#df_dept_salt = df_dept.withColumn("salt", floor(rand(seed = 1)*3))
df_dept_salt = df_dept_salt.withColumn("salted_key", concat_ws("-",df_dept_salt["DEPARTMENT_ID"],df_dept_salt["salt"]))

df_dept_salt.display()


In [0]:
df_emp_dept_join = df_emp_salt.join(df_dept_salt, df_emp_salt.salted_key == df_dept_salt.salted_key, "inner").select(df_emp_salt["EMPLOYEE_ID"], df_emp_salt["FIRST_NAME"], df_emp_salt["LAST_NAME"], df_emp_salt["HIRE_DATE"], df_emp_salt["JOB_ID"], df_emp_salt["SALARY"], df_emp_salt["MANAGER_ID"], df_emp_salt["DEPARTMENT_ID"], df_dept_salt["DEPARTMENT_NAME"])

df_emp_dept_join.display()

In [0]:
df_emp_salt = df_emp_salt.withColumn("FIRST_NAME", regexp_replace(col("FIRST_NAME"),"[^A-Za-z0-9]", ""))
df_emp_salt = df_emp_salt.withColumn("LAST_NAME", regexp_replace(col("LAST_NAME"),"[^A-Za-z0-9]", ""))
df_emp_salt.display()

**UNIT TEST**

In [0]:
df_emp_salt.write.format("delta").mode("overwrite").saveAsTable("Bronze_Nulls_emp")

In [0]:
from pyspark.sql.functions import lit, count, when, col

def count_values(df, column):
    return df_emp_salt.select(
        count(when(col(column).isNull(), column)).alias("Null_count"),
        count(when(col(column).isNotNull(),column)).alias("NotNull_column"),
        count(when(col(column) == "", column)).alias("Blank_Count"),
        count(lit(1)).alias("Total_Rows")
        )
                                                                         

In [0]:
def count_values(df, column):
    return df_emp_salt.select(
        count(when(col(column).isNull(), column)).alias("Null_count"),
        count(when(col(column).isNotNull(),column)).alias("NotNull_column"),
        count(when(col(column) == "", column)).alias("Blank_Count"),
        count(lit(1).alias("Total_Rows"))
        )

In [0]:
display(count_values(df_emp_salt, "FIRST_NAME"))
#display(count_values(df_emp_salt, "DEPARTMENT_ID"))


In [0]:
%sql
select employee_id,first_name, department_id, count(department_id) from bronze_nulls_emp
where department_id | employee_id is null
group by employee_id, first_name,department_id;

In [0]:
%sql
update bronze_nulls_emp 
set employee_id = 300 where first_name = 'Irfan';

In [0]:
%sql
select * from bronze_nulls_emp where employee_id = 300;   


In [0]:
%sql
select count(case when COMMISSION_PCT is null then 1 end) as null_cnt,count(case when COMMISSION_PCT is not null then 1 end) as NOT_NULL
from bronze_nulls_emp;


In [0]:
emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

dept_data = [
    ["101", "Sales", "NYC", "US", "1000000"],
    ["102", "Marketing", "LA", "US", "900000"],
    ["103", "Finance", "London", "UK", "1200000"],
    ["104", "Engineering", "Beijing", "China", "1500000"],
    ["105", "Human Resources", "Tokyo", "Japan", "800000"],
    ["106", "Research and Development", "Perth", "Australia", "1100000"],
    ["107", "Customer Service", "Sydney", "Australia", "950000"]
]

dept_schema = "department_id string, department_name string, city string, country string, budget string"

emp = spark.createDataFrame(data=emp_data, schema=emp_schema)
dept = spark.createDataFrame(data=dept_data, schema=dept_schema)

In [0]:
df_joined = emp.alias("e").join(dept.alias("d"), how="inner", on=emp.department_id==dept.department_id)

df_joined.select("e.name", "d.department_id", "d.department_name", "e.salary").show()

**JOIN STRATERGIES**

In [0]:
display(dbutils.fs.ls("/Volumes/workspace/default/emp/src_DEPT_FF.csv"))
display(dbutils.fs.ls("/Volumes/workspace/default/emp/EMP_FF_SRC.txt"))

In [0]:
df_emp = spark.read.format("csv").option("header"=True).option("inferSchema"==True).option("delimiter", "|").load("/Volumes/workspace/default/emp/EMP_FF_SRC.txt")
df_dept = spark.read.csv("/Volumes/workspace/default/emp/src_DEPT_FF.csv", header=True, inferSchema=True)

df_emp.display()

In [0]:
from pyspark.sql.functions import broadcast

df_broadcast = df_emp.join(broadcast(df_dept), "department_id", "inner")
df_broadcast.show()
    

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

df_emp = spark.read.format("csv").option("header", "true").option("dateFormat", "MM/dd/yyyy").option("inferSchema", "true").option("delimiter", "|").load("/Volumes/workspace/default/emp/EMP_FF_SRC.txt")

df_emp.display()


df_dept = spark.read.format("csv").option("header", "true").option("dateFormat", "MM/dd/yyyy").option("inferSchema", "true").option("delimiter", ",").load("/Volumes/workspace/default/emp/src_DEPT_FF.csv")

df_dept.display()

In [0]:
df_broadcast = df_emp.join(broadcast(df_dept), "department_id", "inner")
df_broadcast.display()
#df_broadcast.cache()
df_broadcast.count()
df_broadcast.explain()



In [0]:
df1 = spark.range(1,100000).withColumnRenamed("id", "id1")
df2 = spark.range(1,100).withColumnRenamed("id", "id2")

result = df1.join(df2, df1.id1 == df2.id2, "inner")
result.explain()

In [0]:
df1 = spark.range(1,100000000).withColumnRenamed("id", "id1")
df2 = spark.range(1,100000000).withColumnRenamed("id", "id2")

result = df1.join(df2, df1.id1 == df2.id2, "inner")
result.explain()

In [0]:
df1 = spark.range(1,1000000000000).withColumnRenamed("id", "id1")
df2 = spark.range(1,100000000000).withColumnRenamed("id", "id2")

result = df1.join(df2, df1.id1 == df2.id2, "inner")
result.explain()